# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

In [0]:
# importar librerias 
from pyspark.sql.functions import col, avg, round, lower, to_date, min, max, unix_timestamp, from_unixtime, regexp_replace


In [0]:
df_room_type.show()  # Muestra las primeras filas para verificar las columnas
df_room_type.printSchema()  # Verifica el esquema de columnas



+----------+--------------------+---------------+
|       _c0|                 _c1|            _c2|
+----------+--------------------+---------------+
|listing_id|         description|      room_type|
|      2595|Skylit Midtown Ca...|Entire home/apt|
|      3831|Cozy Entire Floor...|Entire home/apt|
|      5099|Large Cozy 1 BR A...|Entire home/apt|
|      5178|Large Furnished R...|   private room|
|      5238|Cute & Cozy Lower...|Entire home/apt|
|      5295|Beautiful 1br on ...|Entire home/apt|
|      5441|Central Manhattan...|   Private room|
|      5803|Lovely Room 1, Ga...|   Private room|
|      6021|Wonderful Guest B...|   Private room|
|      6848|Only 2 stops to M...|entire home/apt|
|      7097|Perfect for Your ...|Entire home/apt|
|      7322|     Chelsea Perfect|   Private room|
|      7726|Hip Historic Brow...|Entire home/apt|
|      8024|CBG CtyBGd HelpsH...|   PRIVATE ROOM|
|      8025|CBG Helps Haiti R...|   PRIVATE ROOM|
|      8110|CBG Helps Haiti R...|   Private room|


In [0]:
# Cargar el archivo CSV con datos de precios y ubicaciones
df_price = spark.read.option("header", "true").csv("/FileStore/examenDataBricks/Caso_3/airbnb_price.csv")

# Cargar el archivo Excel con datos sobre los tipos de habitaciones
df_room_type = spark.read.option("delimiter", ";").csv("/FileStore/examenDataBricks/Caso_3/airbnb_room_type.csv")

df_room_type = df_room_type.withColumnRenamed("_c0", "listing_id") \
                           .withColumnRenamed("_c1", "description") \
                           .withColumnRenamed("_c2", "room_type")

# Cargar el archivo TSV con información sobre los anfitriones
df_last_review =  spark.read.option("delimiter", "\t").option("header", "true").csv("/FileStore/examenDataBricks/Caso_3/airbnb_last_review.tsv")

# Unir df_price con df_room_type usando la columna 'listing_id'
df_combined_1 = df_price.join(df_room_type, on="listing_id", how="inner")

# Unir el resultado con df_last_review usando la columna 'listing_id'
df_combined_final = df_combined_1.join(df_last_review, on="listing_id", how="inner")

# Mostrar el DataFrame combinado
df_combined_final.display()


listing_id,price,nbhood_full,description,room_type,host_name,last_review
2595,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt,Jennifer,May 21 2019
3831,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,July 05 2019
5099,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,June 22 2019
5178,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,June 24 2019
5238,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,June 09 2019
5295,135 dollars,"Manhattan, Upper West Side",Beautiful 1br on Upper West Side,Entire home/apt,Lena,June 22 2019
5441,85 dollars,"Manhattan, Hell's Kitchen",Central Manhattan/near Broadway,Private room,Kate,June 23 2019
5803,89 dollars,"Brooklyn, South Slope","Lovely Room 1, Garden, Best Area, Legal rental",Private room,Laurie,June 24 2019
6021,85 dollars,"Manhattan, Upper West Side",Wonderful Guest Bedroom in Manhattan for SINGLES,Private room,Claudio,July 05 2019
6848,140 dollars,"Brooklyn, Williamsburg",Only 2 stops to Manhattan studio,entire home/apt,Allen & Irina,June 29 2019


- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
# 1: ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.

# Obtener las fechas mínima y máxima de las reseñas
min_date = df_combined_final.select(min("last_review")).first()[0]
max_date = df_combined_final.select(max("last_review")).first()[0]

# Mostrar los resultados
print(f"Primera reseña: {min_date}")
print(f"Última reseña: {max_date}")


Primera reseña: April 01 2019
Última reseña: May 31 2019


In [0]:
# 2. ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.

# Hay habitaciones con room_type = PRIVATE ROOM, hacemos lowercase
df_combined_final = df_combined_final.withColumn(
    "room_type", lower(col("room_type"))
)

# Filtrar anuncios con tipo de habitación "Private room"
private_rooms_count = df_combined_final.filter(df_combined_final["room_type"] == "private room").count()

# Mostrar el resultado
print(f"Cantidad de anuncios de habitaciones privadas: {private_rooms_count}")


Cantidad de anuncios de habitaciones privadas: 11282


In [0]:
# 3. ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.

from pyspark.sql import functions as F

# Eliminar la palabra 'dollars' y cualquier espacio extra de la columna 'price'
df_final = df_combined_final.withColumn("price", F.regexp_replace("price", " dollars", "").cast("double"))

# Calcular el precio promedio y redondearlo a dos decimales
avg_price_rounded = df_final.agg(F.round(F.avg("price"), 2).alias("precio_promedio")).collect()[0]["precio_promedio"]

# Mostrar el valor redondeado
print(avg_price_rounded)

None


In [0]:
# 4. Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

from pyspark.sql import Row

# Crear un Row con las variables
data = Row(
    first_reviewed=min_date,
    last_reviewed=max_date,
    nb_private_rooms=private_rooms_count,
    avg_price=avg_price_rounded
)

# Crear un DataFrame a partir de los datos
review_dates = spark.createDataFrame([data])

# Mostrar el DataFrame resultante
review_dates.display()